<a href="https://colab.research.google.com/gist/Srgr0/51d2270f7147c44011da5ae606d55f7a/mmvc-clone-to-googledrive-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMVCのリポジトリをGoogle Driveに保存する

ver.2022/09/06

GitHub上のMMVC_Trainerのリポジトリをクローンし、Google Driveに保存します。

​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。 { display-mode: "form" }

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title ## 2 MMVC_Trainerのクローン
#@markdown 以下で指定したディレクトリ上に、MMVC_Trainerというフォルダが作成されます。

#@markdown 実行前に、画面右上にあるアカウントのアイコンをクリックして、アカウントを確認してください。

#@markdown 成功すると、最後に以下のようなメッセージが表示されます。
#@markdown ```
#@markdown Checking out files: 100% (902/902), done.
#@markdown ```

#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/" #@param {type:"string"}

%cd $directory
!git clone --depth=1 https://github.com/isletennos/MMVC_Trainer/

​

In [ ]:
#@title ## 3 確認
#@markdown 以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```

!ls -1 /content/drive/MyDrive/MMVC_Trainer